In [1]:
%load_ext autoreload
%autoreload 2

In [8]:
from pathlib import Path

from IPython.display import display, JSON
import plotly.graph_objects as go

from openfisca_core.simulation_builder import SimulationBuilder
from openfisca_france import FranceTaxBenefitSystem
from openfisca_france.model.base import TypesStatutOccupationLogement
from taxipp.utils.test_case.utils import create_scenario_superieur_smic

import openfisca_interactive

In [9]:
tax_benefit_system = FranceTaxBenefitSystem()
decomposition = tax_benefit_system.load_decomposition()
tax_benefit_system.add_variables_from_decomposition_tree(decomposition, update=True)

In [10]:
bars = openfisca_interactive.get_bars(decomposition)

In [11]:
period = 2019
test_case_scenario = create_scenario_superieur_smic(year = period, statut_occupation_logement = TypesStatutOccupationLogement.proprietaire.name)

In [15]:
from openfisca_france.scenarios import init_single_entity

simulation_builder = SimulationBuilder()
simulation =  init_single_entity(tax_benefit_system.new_scenario(), **test_case_scenario).new_simulation()
salaire_de_base_arr = simulation.calculate_add("salaire_de_base", period=period)
results = openfisca_interactive.calculate_bars(bars, simulation, period=period)
salaire_index = 2

In [16]:
displayed_bars = list(openfisca_interactive.iter_displayed_bars(
    bars, results, x_variable_arr=salaire_de_base_arr, x_variable=salaire_de_base_arr[salaire_index],
    include_subtotals=False, include_zero=False))

In [17]:
BarType = openfisca_interactive.BarType

waterfall = go.Waterfall()
waterfall.x = [bar.short_name or bar.variable_name for bar in displayed_bars]
waterfall.y = [
    bar.value if bar.type == BarType.VALUE else 0
    for bar in displayed_bars
]
waterfall.measure = [
    "relative" if bar.type == BarType.VALUE else "total"
    for bar in displayed_bars
]
waterfall.text = [
    ("{:+.0f}" if bar.type == BarType.VALUE else "{:.0f}").format(bar.value)
    for bar in displayed_bars
]

layout = go.Layout(title=f"Décomposition du revenu disponible ({period})")
go.FigureWidget(data=[waterfall], layout=layout)

FigureWidget({
    'data': [{'measure': [relative, relative, relative, relative, relative,
                   …